In [1]:
from __future__ import print_function

import SimpleITK as sitk

#%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact, fixed
import os
import gui
import resample_itk_image
from myshow import myshow, myshow3d
from itertools import chain

import pandas as pd

%matplotlib widget

In [39]:
# ubuntu
#data_folder = "/home/djcald/craniotomy_analysis/data"

# PC
data_folder = r"C:\Users\david\OneDrive - UW\craniotomy_analysis_data"
subj_name = "subj" + "1"
data_subj = os.path.join(data_folder, subj_name,"rawdicom")
OUTPUT_DIR = os.path.join(data_folder,subj_name,'output')

In [3]:
# Read the original series. First obtain the series file names using the
# image series reader.
series_IDs = sitk.ImageSeriesReader.GetGDCMSeriesIDs(data_subj)
if not series_IDs:
    print("ERROR: given directory \""+data_folder+"\" does not contain a DICOM series.")
    
series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(data_subj, series_IDs[0])

series_reader = sitk.ImageSeriesReader()
series_reader.SetFileNames(series_file_names)

# Configure the reader to load all of the DICOM tags (public+private):
# By default tags are not loaded (saves time).
# By default if tags are loaded, the private tags are not loaded.
# We explicitly configure the reader to load tags, including the
# private ones.
series_reader.MetaDataDictionaryArrayUpdateOn()
#series_reader.LoadPrivateTagsOn()
image3D = series_reader.Execute()

In [4]:
reader = sitk.ImageFileReader()
reader.SetFileName(series_file_names[0])
#reader.LoadPrivateTagsOn();

reader.ReadImageInformation();

#for k in reader.GetMetaDataKeys():
#    v = reader.GetMetaData(k)
#    print("({0}) = = \"{1}\"".format(k,v))

In [5]:
print('Image Spacing ',image3D.GetSpacing())
print('Image Size ', image3D.GetSize())

orig_size = image3D.GetSpacing()
    


Image Spacing  (0.46904315196998, 0.46875, 2.0003387928009033)
Image Size  (533, 512, 110)


In [6]:
image3D_resample = resample_itk_image.resample_img(image3D,[1,1,1])


In [7]:
# recast types as necessary for analysis 

recast_image3D = sitk.Cast(sitk.RescaleIntensity(image3D_resample), sitk.sitkUInt8)
recast32_image3D = sitk.Cast(image3D_resample, sitk.sitkFloat32)
#
#img = sitk.ReadImage(series_file_names)

In [8]:
z = int(image3D.GetDepth()/2)
plt.imshow(sitk.GetArrayViewFromImage(image3D)[z,:,:], cmap=plt.cm.Greys_r)
plt.axis('off');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
intensity_values = sitk.GetArrayViewFromImage(image3D)
values_list = list(intensity_values)
values_flattened = list(chain.from_iterable(values_list))
values_flattened = np.array(values_flattened).flatten()

plt.figure()
plt.hist(values_flattened, bins=100)
plt.title("Intensity Values in ROI")
plt.show()      

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
point_acquisition_interface = gui.PointDataAquisition(image3D_resample)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
initial_seed_point_indexes = point_acquisition_interface.get_point_indexes()

initial_seed_point_indexes

[(77, 39, 109),
 (145, 28, 109),
 (180, 52, 109),
 (54, 77, 109),
 (161, 146, 109),
 (63, 140, 109)]

In [41]:
#seg_implicit_thresholds = sitk.ConfidenceConnected(image3D, seedList=initial_seed_point_indexes,
#                                                   numberOfIterations=1,
#                                                   multiplier=2,
#                                                   initialNeighborhoodRadius=5,
#                                                   replaceValue=1)

seg_implicit_thresholds = sitk.ConnectedThreshold(image3D_resample, seedList=initial_seed_point_indexes, lower=400, upper=1700)

gui.MultiImageDisplay(image_list = [sitk.LabelOverlay(recast_image3D, seg_implicit_thresholds)],                   
                      title_list = ['confidence connected result'])

C:\Users\david\SharedCode\craniotomy_analysis\gui.py:464: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  self.fig, self.axes = plt.subplots(row_num,col_num,figsize=figure_size)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
myshow3d(seg_implicit_thresholds)

interactive(children=(IntSlider(value=109, description='z', max=219), Output()), _dom_classes=('widget-interac…

In [14]:
len(initial_seed_point_indexes)

6

In [15]:
seg = sitk.Image(image3D_resample.GetSize(), sitk.sitkUInt8)
seg.CopyInformation(image3D_resample)
for index in range(1,len(initial_seed_point_indexes)):
    seg[initial_seed_point_indexes[index]] = 1
seg = sitk.BinaryDilate(seg, 10)

In [16]:
myshow(sitk.LabelOverlay(recast_image3D, seg))

interactive(children=(IntSlider(value=109, description='z', max=219), Output()), _dom_classes=('widget-interac…

In [17]:
#stats = sitk.LabelStatisticsImageFilter()
#stats.Execute(image3D, seg)

#factor = 3.5
#lower_threshold = stats.GetMean(1)-factor*stats.GetSigma(1)
#upper_threshold = stats.GetMean(1)+factor*stats.GetSigma(1)
#print(stats)

lower_threshold = 300
upper_threshold = 1700
print(lower_threshold,upper_threshold)

300 1700


In [18]:
init_ls = sitk.SignedMaurerDistanceMap(seg, insideIsPositive=True, useImageSpacing=True)

In [19]:
lsFilter = sitk.ThresholdSegmentationLevelSetImageFilter()
lsFilter.SetLowerThreshold(lower_threshold)
lsFilter.SetUpperThreshold(upper_threshold)
lsFilter.SetMaximumRMSError(0.005)
lsFilter.SetNumberOfIterations(1000)
lsFilter.SetCurvatureScaling(1)
lsFilter.SetPropagationScaling(1)
lsFilter.ReverseExpansionDirectionOn()

#niter = 0
#for i in range(0,10):
#    ls = lsFilter.Execute(init_ls,recast32_image3D)
#    niter += lsFilter.GetNumberOfIterations()
#    t = "LevelSet after "+str(niter)+" iterations and RMS "+str(lsFilter.GetRMSChange())
#    fig = myshow(sitk.LabelOverlay(recast_image3D, ls > 0),title=t)
ls = lsFilter.Execute(init_ls,recast32_image3D)    
print(lsFilter)

itk::simple::ThresholdSegmentationLevelSetImageFilter
  LowerThreshold: 300
  UpperThreshold: 1700
  MaximumRMSError: 0.005
  PropagationScaling: 1
  CurvatureScaling: 1
  NumberOfIterations: 1000
  ReverseExpansionDirection: 1
  ElapsedIterations: 1000
  RMSChange: 0.00915952
  Debug: 0
  NumberOfThreads: 12
  Commands: (none)
  ProgressMeasurement: 1
  ActiveProcess: (none)



In [20]:
myshow(sitk.LabelOverlay(recast_image3D, ls>0))

interactive(children=(IntSlider(value=109, description='z', max=219), Output()), _dom_classes=('widget-interac…

In [21]:
ls_thresh = ls>0

In [22]:
myshow3d(ls_thresh)

interactive(children=(IntSlider(value=109, description='z', max=219), Output()), _dom_classes=('widget-interac…

In [23]:
ls_thresh

<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x00000213818DCAB0> >

In [24]:
#vectorRadius=(1,1,1)
#kernel=sitk.sitkBall
#ls_clean = sitk.BinaryMorphologicalClosing(sitk.Cast(ls,sitk.sitkUInt32),vectorRadius,kernel)

In [25]:
#gui.MultiImageDisplay(image_list = [sitk.LabelOverlay(recast_image3D, ls>0), 
#                                sitk.LabelOverlay(recast_image3D, ls_clean>0)], 
#                  shared_slider=True,
#                  title_list = ['before morphological closing', 'after morphological closing'])

In [26]:
ls_thresh_resize = resample_itk_image.resample_img(ls_thresh,orig_size,True)
recast_ls = sitk.Cast(ls_thresh_resize,sitk.sitkUInt8)

In [27]:
shape_stats = sitk.LabelShapeStatisticsImageFilter()
shape_stats.ComputeOrientedBoundingBoxOn()
shape_stats.Execute(recast_ls)
shape_stats.GetPhysicalSize(1)

773403.1518097897

In [40]:
# Write the image.
output_file_name_3D = os.path.join(OUTPUT_DIR, 'subj1Filt.nii.gz')
sitk.WriteImage(ls_thresh_resize, output_file_name_3D)

In [30]:
5.219e5 - shape_stats.GetPhysicalSize(1)

-251503.1518097897

In [31]:
OUTPUT_DIR

'C:\\Users\\david\\OneDrive - UW\\craniotomy_analysis_data\\subj1\\output'